# Creating a TorchModel
Base models built in PyTorch are already supported by LUME-model. We demonstrate how to create and execute a `TorchModel` below.

In [1]:
import torch

from lume_model.models import TorchModel, TorchModule
from lume_model.variables import ScalarVariable

## Building a Model from Scratch
Instantiation of a `TorchModel` requires specification of the base model (`torch.nn.Module`) and in-/output variables.

In [2]:
# exemplary model definition
base_model = torch.nn.Sequential(
    torch.nn.Linear(2, 1),
)
# variable specification
input_variables = [
    ScalarVariable(name="input1", default_value=0.1, value_range=[0.0, 1.0]),
    ScalarVariable(name="input2", default_value=0.2, value_range=[0.0, 1.0]),
]
output_variables = [
    ScalarVariable(name="output"),
]

# creation of TorchModel
example_model = TorchModel(
    model=base_model,
    input_variables=input_variables,
    output_variables=output_variables,
    fixed_model=True,
)

## Loading a Model from File

An already created model can be saved to a YAML file by calling the `dump` method. The model can then be loaded by simply passing the file to the constructor.

In [4]:
torch_model = TorchModel("../tests/test_files/california_regression/torch_model.yml")
print(torch_model.yaml())

model_class: TorchModel
input_variables:
  MedInc:
    variable_class: ScalarVariable
    default_value: 3.7857346534729004
    value_range: [0.4999000132083893, 15.000100135803223]
    is_constant: false
  HouseAge:
    variable_class: ScalarVariable
    default_value: 29.282135009765625
    value_range: [1.0, 52.0]
    is_constant: false
  AveRooms:
    variable_class: ScalarVariable
    default_value: 5.4074907302856445
    value_range: [0.8461538553237915, 141.90908813476562]
    is_constant: false
  AveBedrms:
    variable_class: ScalarVariable
    default_value: 1.1071722507476807
    value_range: [0.375, 34.06666564941406]
    is_constant: false
  Population:
    variable_class: ScalarVariable
    default_value: 1437.0687255859375
    value_range: [3.0, 28566.0]
    is_constant: false
  AveOccup:
    variable_class: ScalarVariable
    default_value: 3.035413980484009
    value_range: [0.692307710647583, 599.7142944335938]
    is_constant: false
  Latitude:
    variable_class: Sc

## Model Execution and TorchModule
Calling the `evaluate` method allows for model execution on dictionary input. Additionally, instances of `TorchModel` can also be wrapped in a `TorchModule` which is a subclass of `torch.nn.Module`. This allows for seamless integration with `PyTorch` based packages like [BoTorch](https://botorch.org/) and [Xopt](https://christophermayes.github.io/Xopt/).

In [5]:
# generate exemplary input
input_dict = torch_model.random_input(n_samples=1)
input_dict

{'MedInc': tensor([5.4826]),
 'HouseAge': tensor([41.6984]),
 'AveRooms': tensor([40.1493]),
 'AveBedrms': tensor([11.8220]),
 'Population': tensor([9973.0526]),
 'AveOccup': tensor([396.3613]),
 'Latitude': tensor([38.8324]),
 'Longitude': tensor([-121.1657])}

In [6]:
# execute TorchModel
torch_model.evaluate(input_dict)

{'MedHouseVal': tensor(2.5230)}

In [7]:
# wrap in TorchModule
torch_module = TorchModule(model=torch_model)

In [8]:
# execute TorchModule
input_tensor = torch.tensor(
    [input_dict[k] for k in torch_module.input_order]
).unsqueeze(0)
torch_module(input_tensor)

tensor(2.5230)